In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc=SparkContext()

In [3]:
import json

In [4]:
rdd=sc.textFile("/home/deepika/Downloads/netflix_titles.json")

In [5]:
from pyspark.storagelevel import StorageLevel
rdd.persist(StorageLevel.MEMORY_ONLY)

/home/deepika/Downloads/netflix_titles.json MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [6]:
data= rdd.map(lambda x: json.loads(x))

In [7]:
data.take(1)

[{'show_id': 's1',
  'type': 'TV Show',
  'title': '3%',
  'director': '',
  'cast': 'João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi',
  'country': 'Brazil',
  'date_added': 'August 14, 2020',
  'release_year': '2020',
  'rating': 'TV-MA',
  'duration': '4 Seasons',
  'listed_in': 'International TV Shows, TV Dramas, TV Sci-Fi & Fantasy',
  'description': 'In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor.'}]


# how many titles actor/actress appeared in?

In [8]:
titles= data.flatMap(lambda x: (x['cast'].split(",")))

In [9]:
gap=titles.filter(lambda x: x !='')

In [11]:
result= gap.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)

In [40]:
result.collect()

[(' Vaneza Oliveira', 1),
 (' Viviane Porto', 1),
 (' Mel Fronckowiak', 1),
 (' Celso Frateschi', 1),
 (' Héctor Bonilla', 1),
 (' Octavio Michel', 1),
 (' Carmen Beato', 3),
 ('Tedd Chan', 1),
 (' Henley Hii', 5),
 (' Lawrence Koh', 1)]

In [13]:
import os
os.environ["PYTHONHASHSEED"]=str(232)

In [14]:
result.lookup(' Héctor Bonilla')

[]

# How many movies were released in 2008?

In [15]:
movies=data.filter(lambda x:x['release_year']=='2008')

In [16]:
movies.count()

125

In [42]:
movies.collect()

[{'show_id': 's5',
  'type': 'Movie',
  'title': '21',
  'director': 'Robert Luketic',
  'cast': 'Jim Sturgess, Kevin Spacey, Kate Bosworth, Aaron Yoo, Liza Lapira, Jacob Pitts, Laurence Fishburne, Jack McGee, Josh Gad, Sam Golzari, Helen Carey, Jack Gilpin',
  'country': 'United States',
  'date_added': 'January 1, 2020',
  'release_year': '2008',
  'rating': 'PG-13',
  'duration': '123 min',
  'listed_in': 'Dramas',
  'description': 'A brilliant group of students become card-counting experts with the intent of swindling millions out of Las Vegas casinos by playing blackjack.'},
 {'show_id': 's10',
  'type': 'Movie',
  'title': '1920',
  'director': 'Vikram Bhatt',
  'cast': 'Rajneesh Duggal, Adah Sharma, Indraneil Sengupta, Anjori Alagh, Rajendranath Zutshi, Vipin Sharma, Amin Hajee, Shri Vallabh Vyas',
  'country': 'India',
  'date_added': 'December 15, 2017',
  'release_year': '2008',
  'rating': 'TV-MA',
  'duration': '143 min',
  'listed_in': 'Horror Movies, International Movies,

# List all the movies whose duration is greater than 100 mins ?

In [18]:
pairs = data.map(lambda x: (int(x['duration'].split(" ")[0]), x['title']))

In [38]:
pairs.collect()

[(4, '3%'),
 (93, '7:19'),
 (78, '23:59'),
 (80, '9'),
 (123, '21'),
 (1, '46'),
 (95, '122'),
 (119, '187'),
 (118, '706'),
 (143, '1920')]

In [20]:
duration_above_hundred= pairs.filter(lambda x: x[0] >100)

In [37]:
duration_above_hundred.take(10)

['21',
 '187',
 '706',
 '1920',
 '1922',
 'Oct-01',
 '22-Jul',
 '15-Aug',
 '\u200b\u200bKuch Bheege Alfaaz',
 '\u200bGoli Soda 2']

In [22]:
duration_above_hundred.count()

2430

In [23]:
duration_above_hundred=duration_above_hundred.map(lambda x: x[1])

In [36]:
duration_above_hundred.collect()

['21',
 '187',
 '706',
 '1920',
 '1922',
 'Oct-01',
 '22-Jul',
 '15-Aug',
 '\u200b\u200bKuch Bheege Alfaaz',
 '\u200bGoli Soda 2']

# List movies played by “Kareena Kapoor” ?

In [25]:
kareena = data.map(lambda x: (x['cast'].split(",")[0], x['title']))

In [35]:
kareena.collect()

[('João Miguel', '3%'),
 ('Demián Bichir', '7:19'),
 ('Tedd Chan', '23:59'),
 ('Elijah Wood', '9'),
 ('Jim Sturgess', '21'),
 ('Erdal Beşikçioğlu', '46'),
 ('Amina Khalil', '122'),
 ('Samuel L. Jackson', '187'),
 ('Divya Dutta', '706'),
 ('Rajneesh Duggal', '1920')]

In [27]:
kareena1=[]
kareenaa= kareena.groupByKey()
for t in kareenaa.collect():
    kareena1.append((t[0],[i for i in t[1]]))

In [29]:
movie_list_by_kareena_kapoor=sc.parallelize(kareena1)

In [30]:
movie_list_by_kareena_kapoor= movie_list_by_kareena_kapoor.filter(lambda x: x[0]!='')

In [34]:
movie_list_by_kareena_kapoor.collect()

[('Tedd Chan', ['23:59']),
 ('Jim Sturgess', ['21', 'Kidnapping Mr. Heineken']),
 ('Amina Khalil', ['122']),
 ('Omar Epps', ['3022']),
 ('Sadiq Daba', ['Oct-01']),
 ('Anders Danielsen Lie', ['22-Jul']),
 ('Rahul Pethe', ['15-Aug']),
 ('Soumitra Chatterjee', ['\u200bMayurakshi', 'Peace Haven']),
 ('Yoo Ah-in', ['#Alive', 'Burning', 'Chicago Typewriter']),
 ('Helen Mirren',
  ['#AnneFrank - Parallel Stories',
   "Legend of the Guardians: The Owls of Ga'Hoole",
   'The Hundred-Foot Journey',
   'Winchester'])]

In [32]:
movie_list_by_kareena_kapoor.lookup("Kareena Kapoor")

[['Chameli', 'Ek Main Aur Ekk Tu', 'Heroine', 'Kurbaan', 'Satyagraha']]

In [46]:
movie_list_by_kareena_kapoor.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/movie_list_by_kareena_kapoor/")

In [47]:
duration_above_hundred.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/duration_above_hundred/")

In [48]:
movies.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/movies_in_2008/")

In [49]:
result.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/titles_actor_actress_appeared/")